In [1]:
import os
import sys
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
from tensorflow import feature_column as fc
plt.rcParams["font.family"] = 'NanumBarunGothic'
TENSORBOARD_BINARY = '/home/hoondori/anaconda3/envs/ai/bin/tensorboard'
os.environ['TENSORBOARD_BINARY'] =  TENSORBOARD_BINARY
%load_ext tensorboard

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 첫 번째 GPU만 사용하도록 제한
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
        print(e)


# TF2 로 Resnet 자체 구현 및 성능 측정 

* 참고 자료 
  * https://towardsdatascience.com/understand-and-implement-resnet-50-with-tensorflow-2-0-1190b9b52691
  * https://github.com/calmisential/TensorFlow2.0_ResNet


# (baseline) Small CNN CIFAR10

In [2]:
def cnn_cifar10(log_dir):

    #define the convnet 
    def build(input_shape, classes):
        model = models.Sequential() 
        model.add(layers.Convolution2D(32, (3, 3), activation='relu',
                                input_shape=input_shape))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.25)) 

        model.add(layers.Flatten())
        model.add(layers.Dense(512, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(classes, activation='softmax'))
        return model

    # CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
    IMG_CHANNELS = 3
    IMG_ROWS = 32
    IMG_COLS = 32

    #constant
    BATCH_SIZE = 128
    EPOCHS = 20
    CLASSES = 10
    VERBOSE = 1
    VALIDATION_SPLIT = 0.2
    OPTIM = tf.keras.optimizers.RMSprop()

    # data: shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
    # normalize
    X_train, X_test = X_train / 255.0, X_test / 255.0
    # convert to categorical
    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
    y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

    model=build((IMG_ROWS, IMG_COLS, IMG_CHANNELS), CLASSES)
    model.summary()

    # use TensorBoard, princess Aurora!
    callbacks = [
      # Write TensorBoard logs to `./logs` directory
      tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    ]

    # train
    model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
        metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=BATCH_SIZE,
        epochs=EPOCHS, validation_split=VALIDATION_SPLIT, 
        verbose=VERBOSE, callbacks=callbacks) 
    score = model.evaluate(X_test, y_test,
                         batch_size=BATCH_SIZE, verbose=VERBOSE)
    print("\nTest score:", score[0])
    print('Test accuracy:', score[1])

!rm -rf /tmp/logs/cnn_cifar10     
cnn_cifar10('/tmp/logs/cnn_cifar10')  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7200)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3686912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

# Resnet-50 구현 

In [4]:
#### Necessary Imports for Neural Net 

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

def res_identity(x, filters):

    #  x => 1x1xf1 => 3x3xf1 => 1x1xf2 => x + residual 
    
    x_skip = x # for residual learning
    f1, f2 = filters 
    
    # first block (1x1xf1 s1)
    x = Conv2D(f1, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    # second block (3x3xf1 s1)
    x = Conv2D(f1, kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    # third block (1x1xf2 s1)
    x = Conv2D(f2, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # add the input
    x = Add()([x, x_skip])
    
    return x

def res_conv(x, s, filters):
    # 첫번째 block에서 s>1이면 x 와 residual 경로상의 크기가 달라지므로 이를 신경써서 맞추어 주어야 한다.
    # 해법은(ex. s=2) residual 로 오는쪽에서 1x1 s=2로 보폭을 늘리고 로 w/h를 반만큼 줄이고, output channel을 2배로 늘려야 한다.
    
    
    #  x => 1x1xf1 s2 => 3x3xf1 s1=> 1x1xf2 s1=> x + residual( by 1x1xf2 s2) 
    
    x_skip = x
    f1, f2 = filters
    
    # first block (1x1xf1 s>1)
    x = Conv2D(f1, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    # second block (3x3xf1 s1)
    x = Conv2D(f1, kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    # third block (1x1xf2 s1)
    x = Conv2D(f2, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    # skip connection 
    x_skip = Conv2D(f2, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)
    
    return x

def resnet50(input_shape, n_class):

    input_im = Input(shape=input_shape) # cifar 10 images size
    x = ZeroPadding2D(padding=(3, 3))(input_im)

    # 1st stage
    # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #2nd stage 
    # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))

    # 3rd stage

    x = res_conv(x, s=2, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))

    # 4th stage

    x = res_conv(x, s=2, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))

    # 5th stage

    x = res_conv(x, s=2, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))

    # ends with average pooling and dense connection

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(n_class, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

    # define the model 

    model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model


In [5]:
# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
EPOCHS = 20
CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = tf.keras.optimizers.RMSprop()

log_dir = '/tmp/logs/resnet50'

# data: shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
# normalize
X_train, X_test = X_train / 255.0, X_test / 255.0
# convert to categorical
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, CLASSES)

model=resnet50((IMG_ROWS, IMG_COLS, IMG_CHANNELS), CLASSES)
#model.summary()

# use TensorBoard, princess Aurora!
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
    metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=BATCH_SIZE,
    epochs=EPOCHS, validation_split=VALIDATION_SPLIT, 
    verbose=VERBOSE, callbacks=callbacks) 
score = model.evaluate(X_test, y_test,
                     batch_size=BATCH_SIZE, verbose=VERBOSE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])


Epoch 1/20
313/313 [==============================] - 18s 59ms/step - loss: 10.5917 - accuracy: 0.2984 - val_loss: 6.9804 - val_accuracy: 0.1227
Epoch 2/20
313/313 [==============================] - 16s 53ms/step - loss: 5.2936 - accuracy: 0.3489 - val_loss: 22.7027 - val_accuracy: 0.1063
Epoch 3/20
313/313 [==============================] - 16s 52ms/step - loss: 4.5113 - accuracy: 0.3478 - val_loss: 213.0576 - val_accuracy: 0.1683
Epoch 4/20
313/313 [==============================] - 16s 53ms/step - loss: 3.6823 - accuracy: 0.4129 - val_loss: 3.7933 - val_accuracy: 0.2173
Epoch 5/20
313/313 [==============================] - 16s 53ms/step - loss: 3.3536 - accuracy: 0.4340 - val_loss: 3.6055 - val_accuracy: 0.3927
Epoch 6/20
313/313 [==============================] - 17s 53ms/step - loss: 3.1226 - accuracy: 0.4461 - val_loss: 3.6890 - val_accuracy: 0.2319
Epoch 7/20
313/313 [==============================] - 17s 53ms/step - loss: 2.7756 - accuracy: 0.4688 - val_loss: 23.5130 - val_accu